# **iFLYTEK叶菜病虫害图像识别挑战赛**

## **赛事链接：**  http://challenge.xfyun.cn/topic/info?type=pests-diseases

## **目前排名：**![](https://ai-studio-static-online.cdn.bcebos.com/b30cb7de00b046858ea8a52b8a4d9461b6b4f64909c345d491aa94210492d515)


## **一、赛事背景**
农作物病虫害严重制约着农业生产，因为农作物病虫害种类多、密度大，极易造成农作物大量减产。同时由于传统人眼识别病虫害的方法速度较慢、准确度较低，会导致农药的滥用，破坏自然环境。如今随着精准农业和智慧农业概念的兴起和发展，利用信息技术辅助农业生产，实现对农作物病虫害的智能识别和检测，以减少不必要的农药喷施，对保护生态系统均衡，保障农作物安全生产，提高农作物的质量方面，有着十分重要的促进作用。

## **二、赛事任务**
最为有效的病虫害识别方法是图片识别，本次大赛提供了大量农民在田间地头拍摄的叶菜的病虫害图片，参赛选手需基于提供的样本构建模型，实现叶菜的病虫害图像识别，即为图片分类。

## **三、评审规则**

评估指标
本模型依据提交的结果文件，采用准确率进行评价。

![](https://ai-studio-static-online.cdn.bcebos.com/7d526a6198d94a6181501eb0651292480854c35fbbe64c5e95cbb5a1185b32a1)

## **四、数据说明**

1.数据说明


本次比赛为参赛选手提供了叶菜的病虫害图像数据：包括图像及其所属病虫害标签。数据主体为农民在不同环境条件下拍摄的叶菜农作物图像，每张图像的主体突出度，背景复杂程度、光照条件，图像清晰度均存在一定差别。图片已按类别放在不同文件夹内，文件夹名称即为图片的category_id。

1：用药不当

![](https://ai-studio-static-online.cdn.bcebos.com/9a2373152d3345748cbcdc99ea2b4f7e6dd03debf8f040a8a26008773dbdc557)


2：疫病

![](https://ai-studio-static-online.cdn.bcebos.com/d6378a23e4714a1aa93b84a8b030845e20c3350a7f714e7d90983a8b5321092b)


3：炭疽病

![](https://ai-studio-static-online.cdn.bcebos.com/a2b0bedd36154d76a84e3692d2a80b76787f92383e494c5baf3ec0992e562e87)



本次比赛为参赛选手提供的数据分为训练集、测试集、提交样例三类文件：

训练集：包含1000多张。

测试集：包含300张，图片文件的名称即为image_id。

提交样例：表头为image_id和category_id的CSV文件，选手提交数据时需要将测试集的图片id与模型预测的类别id按样例格式填入CSV中，进行提交。


# **解题思路：**

1.首先做一些数据的统计以及数据探索

2.然后进行选择合适的模型进行构造baseline

3.准确无误输出结果并对其进行提交成功

4.后期对模型进行修改

5.对模型进行融合


# **一、解压数据**

首先解压压缩包，然后把里面的train.csv文件复制出来，然后把所有类别的图片都放在train这个文件夹下面

In [ ]:
!unzip data/data98942/leaf.zip

In [ ]:
!cp trian1/train2/train.csv -d ./

In [ ]:
!cp trian1/train2/1/* -d ./train/
!cp trian1/train2/2/* -d ./train/
!cp trian1/train2/3/* -d ./train/

## **数据EDA**

  探索性数据分析（Exploratory Data Analysis，简称EDA），是指对已有的数据（原始数据）进行分析探索，通过作图、制表、方程拟合、计算特征量等手段探索数据的结构和规律的一种数据分析方法。一般来说，我们最初接触到数据的时候往往是毫无头绪的，不知道如何下手，这时候探索性数据分析就非常有效。

对于图像分类任务，我们通常首先应该统计出每个类别的数量，查看训练集的数据分布情况。通过数据分布情况分析赛题，形成解题思路。（洞察数据的本质很重要。）

In [2]:
import pandas as pd
df = pd.read_csv('train.csv')
d=df['label'].hist().get_figure()
d.savefig('1.jpg')

![](https://ai-studio-static-online.cdn.bcebos.com/8d2dc0ff48994846b5feb6fdde98244bdcc768221e1242f8ab0bf7f52264b512)


# **二、按比例划分数据集**

**用pandas模块读取train.csv信息，即图像的名称以及对应的标签**

**再按照8：2的比列生成训练集和验证集，并且保存到work/文件夹下**

In [ ]:
import pandas as pd
import codecs
import os
from PIL import Image
import numpy as np
import random

df = pd.read_csv('train.csv')


from sklearn.utils import shuffle
df = shuffle(df)#随机打乱顺序

all_file_dir = 'work'

train_file = codecs.open(os.path.join(all_file_dir, "train_list.txt"), 'w')#设置训练集的地址
eval_file = codecs.open(os.path.join(all_file_dir, "eval_list.txt"), 'w')#设置验证集的地址

# df = df.values
# random.shuffle(df)
print(df)
# df.sample(frac=1)

image_path_list = df['image'].values
label_list = df['label'].values
# 划分训练集和校验集
all_size = len(image_path_list)
train_size = int(all_size * 0.8)#设置训练集数目
train_image_path_list = image_path_list[:train_size]
train_label_list = label_list[:train_size]
val_image_path_list = image_path_list[train_size:]
val_label_list = label_list[train_size:]

image_path_pre = 'train'

for file,label_id in zip(train_image_path_list, train_label_list):
    # print(file)
    # print(label_id)
    try:
        img = Image.open(os.path.join(image_path_pre, file))
        
        # train_file.write("{0}\0{1}\n".format(os.path.join(image_path_pre, file), label_id))
        train_file.write("{0}{1}{2}\n".format(file,' ', label_id))
        # eval_file.write("{0}\t{1}\n".format(os.path.join(image_path_pre, file), label_id))
    except Exception as e:
        pass
        # 存在一些文件打不开，此处需要稍作清洗
        # print('error!')

for file,label_id in zip(val_image_path_list, val_label_list):
    # print(file)
    # print(label_id)
    try:
        img = Image.open(os.path.join(image_path_pre, file))
        # train_file.write("{0}\t{1}\n".format(os.path.join(image_path_pre, file), label_id))
        eval_file.write("{0}{1}{2}\n".format(file,' ', label_id))
    except Exception as e:
        # pass
        # 存在一些文件打不开，此处需要稍作清洗
        # print('error!')

train_file.close()
# #需要稍作清洗
# print('error!')

train_file.close()
eval_file.close()

                                        image  label
318  13587eaf-e742-4fa6-b92b-f9c4d309ef94.jpg      2
84   4D29D1B2-3C84-49F8-BD89-A1757382CC03.png      1
978  a8c7debf-4024-481a-8afc-ce0dd69f7f52.jpg      3
292  03e9b85a-5847-4e5d-858f-b2eb6d60f192.jpg      2
718  14c524e0-45f4-486f-a960-e003d3409845.jpg      3
..                                        ...    ...
526  9c3c86b4-bf41-4159-90f3-ab4866cdd2e4.jpg      2
576  bd04bb79-75fb-431e-9ec7-72de35f6ab0c.jpg      2
864  65264c87-79a5-48ba-9333-27ddeca86960.jpg      3
815  45c5e5d3-4b1b-4c2e-9465-bd67d84c122d.jpg      3
134  7b5fbed3-4c5e-4c1d-9698-ec24d1890b29.jpg      1

[1103 rows x 2 columns]


训练集文件

![](https://ai-studio-static-online.cdn.bcebos.com/fde5aa43e4a54666a304b43fa71d10f0a3bea49d8ce045bb819cf636dd341419)


验证集文件

![](https://ai-studio-static-online.cdn.bcebos.com/a331cb760fe248d495ed81c869bbd8cb32b4196cfd9e4e7a9aa90e69dde77da9)


# **三、解压PaddleClas-release-2.1包**

In [ ]:
!unzip data/data98136/PaddleClas-release-2.1.zip

## **下载ResNet50_vd_pretrained预训练权重**

In [ ]:
!mkdir pretrained 
!cd pretrained && wget https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/ResNet50_vd_pretrained.pdparams

mkdir: cannot create directory ‘pretrained’: File exists
--2021-07-08 21:14:11--  https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/ResNet50_vd_pretrained.pdparams
Resolving paddle-imagenet-models-name.bj.bcebos.com (paddle-imagenet-models-name.bj.bcebos.com)... 182.61.200.195, 182.61.200.229, 2409:8c00:6c21:10ad:0:ff:b00e:67d, ...
Connecting to paddle-imagenet-models-name.bj.bcebos.com (paddle-imagenet-models-name.bj.bcebos.com)|182.61.200.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155324854 (148M) [application/octet-stream]
Saving to: ‘ResNet50_vd_pretrained.pdparams’

ResNet50_vd_pretrai 100%[===================>] 148.13M  58.0MB/s    in 2.6s    

2021-07-08 21:14:14 (58.0 MB/s) - ‘ResNet50_vd_pretrained.pdparams’ saved [155324854/155324854]



# **四、执行训练**

本次使用的是使用的ResNet50_vd模块,路径放在configs/ResNet/ResNet50_vd.yaml，下面试对这个配置文件进行说明要更改的地方：

![](https://ai-studio-static-online.cdn.bcebos.com/cf130eadb2cf431dbc9de41149894b479f2f25032ec44f189d4040d0e68e75f2)

pretrained_model: 加载预训练模型，"/home/aistudio/pretrained/ResNet50_vd_pretrained"

model_save_dir: 存放地址，"/home/aistudio/output/"

classes_num: 3，要识别的类别总数

total_images: 1103，照片总数=训练集+验证集

save_interval: 5   隔多少epochs保存一次权重

validate: True     开启验证

valid_interval: 1    每个多少epochs验证一次模型


使用mix数据增强

use_mix: True

ls_epsilon: 0.1

学习率调整

LEARNING_RATE:
    function: 'Cosine'   
    
    params:                   
        lr: 0.001  
        
优化器

OPTIMIZER:

    function: 'Momentum'
    
    params:
        momentum: 0.9
        
    regularizer:
        function: 'L2'
        factor: 0.000070
        
        
 ...........................       

![](https://ai-studio-static-online.cdn.bcebos.com/53439d195b3e4b7abbe23f2ee1bd402cb1bd9b8957854f26b90fa3b777c8bf09)

**训练过程**进行数据增强的过程，这里使用了

DecodeImage（）

RandCropImage（随机裁剪）

RandFlipImage（随机翻转）

NormalizeImage（归一化）

还有Mix（照片拼接）

数据增强还有许多办法，大家可以多多尝试，比如加入随机噪声，对比度的调整等。

**验证过程**则使用了

DecodeImage（）

ResizeImage（重新设置照片大小）

RandCropImage（随机裁剪）

NormalizeImage（归一化）

In [17]:
cd PaddleClas-release-2.1

/home/aistudio/PaddleClas-release-2.1


In [9]:
!python tools/train.py  -c configs/ResNet/ResNet50_vd.yaml

2epoch精度就已经0.44344了，准确率还是可以的，后面我就没去再训练了，只是给大家做了一个演示。
![](https://ai-studio-static-online.cdn.bcebos.com/a0e7317020134b5282b535db48807506366bdc6d1fdb48ceabed093a811e7999)


## **Tips:**

后期训练策略可以是用精度更高的网络文件配置，并且对它进行相对应的调整

例如调整学习率Lr，以及schedule_LR，还有符合模型的数据增强

最后可以进行模型融合或者结果融合等。


# **五、模型评估**

In [18]:
!python -u tools/eval.py -c configs/ResNet/ResNet50_vd.yaml \
-o weights=output/ResNet50_vd/best_model/ppcls.pdparams

# **六、进行预测并生成结果文件**

结果文件为submisson.csv，生成后便可提交。

In [11]:
!python PaddleClas-release-2.1/tools/infer/infer.py \
    -i test  \
    --model ResNet50_vd \
    --pretrained_model "output/ResNet50_vd/best_model/ppcls" \
    --load_static_weights False \
    --class_num=3

这里我对PaddleClas-release-2.1/tools/infer/infer.py进行了一点修改

因为直接用infer.py输出的结果是这种格式 File:{}, Top-{} result: class id(s): {}, score(s): {}。

我们可以看到我们要提交的文件格式是image_id以及category_id，也就是对应这里的File以及class id。

所以我在85行插入了如下代码，就是把读取到的文件信息以及其标签信息进行存放，然后利用pandas模块把它制作成表格，形成可提交文件。

![](https://ai-studio-static-online.cdn.bcebos.com/07cbbcc1a70d483bb7fa2c422a82373311404f71153f455dbc0b9c4f9c4f963f)


![](https://ai-studio-static-online.cdn.bcebos.com/6da767a941874845a5648ddb2aa06c6ca19158df571c455bbe1c6a0074233a41)


## **生成结果文件**


![](https://ai-studio-static-online.cdn.bcebos.com/cc941f5b189a421ea26e2e448012c081c35afbe7d6ae4c44a608ea589c85009a)




# **总结**

针对图像分类比赛,大家在选定了一个baseline之后可以尝试各种技巧，包括学习率调整策略，模型调参等等。关于图像分类竞赛的一些技巧，大家可以去网上搜索相关的trick。

建议

刚开始的小白，比赛优先使用简单的模型（如ResNet50），快速跑完整个训练和预测流程。

要有一定毅力和耐心，不怕失败，比赛过程往往会遇到很多预想不到的问题。另外数据扩增方法一定要反复尝试，会很大程度上影响模型精度。

后期上分可使用模型融合

# **关于更多关于PaddleClas的信息请参考下面的链接地址。**

PaddleClas教程文档地址：https://github.com/PaddlePaddle/PaddleClas

PaddleClas Github地址：https://github.com/PaddlePaddle/PaddleClas


# **关于作者**

感兴趣的方向为：目标检测，图像分类等

AIstudio主页： 我在AI Studio上获得白银等级，点亮3个徽章，来互关呀~ https://aistudio.baidu.com/aistudio/personalcenter/thirdview/474269

Github主页： https://github.com/Niki173

欢迎大家有问题留言交流学习，共同进步成长。